# Modules

In [1]:
import json
import os
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import re
import requests

from bs4 import BeautifulSoup as bs

# Constantes

In [2]:
PATH_DATA = r"../Data"
PATH_DATA_RAW = rf"{PATH_DATA}\Citya\Raw"

# Application

In [3]:
if os.path.exists(PATH_DATA):
    print('Le dossier Data existe déja.')
else: 
    os.makedirs(PATH_DATA)
    print('Le dossier Data a été créé avec succès.')

Le dossier Data existe déja.


In [4]:
if os.path.exists(PATH_DATA_RAW):
    print('Le dossier Data Raw existe déja.')
else: 
    os.makedirs(PATH_DATA_RAW)
    print('Le dossier Data Raw a été créé avec succès.')

Le dossier Data Raw existe déja.


In [5]:
if os.path.exists(PATH_DATA_RAW + "/Citya/json"):
    print('Le dossier Data Raw citya existe déja.')
else: 
    os.makedirs(PATH_DATA_RAW + "/Citya/json")
    print('Le dossier Data Raw citya a été créé avec succès.')
    
if os.path.exists(PATH_DATA_RAW + "/Citya/parquet"):
    print('Le dossier Data Raw citya existe déja.')
else: 
    os.makedirs(PATH_DATA_RAW + "/Citya/parquet")
    print('Le dossier Data Raw citya a été créé avec succès.')

Le dossier Data Raw citya a été créé avec succès.
Le dossier Data Raw citya a été créé avec succès.


## Téléchargement données d'une page

In [6]:
page = 1
url = fr"https://www.citya.com/annonces/vente/appartement,maison?sort=b.dateMandat&direction=desc&page={page}"
response = requests.get(url)

html = response.content
soup = bs(html, "html")
# print("Status code: ", response.status_code)
descs = soup.find_all('div', {'class': "infos"})
# print("DESCS: ", descs)

data_citya = []
    

for i,des in enumerate(descs):
    content = {'id': i,
                'title': None,
                'description': None,
                'type': None,
                'surface': None,
                'price': None,
                'city': None,
                'postal_code': None,
                'number_pieces': None,
                }
    
    # print("DES: ", des)
    content['title'] = des.find('h3').get_text().strip().replace('\n','')
    
    
    content['description'] = des.find('p', class_="description-start").get_text().strip()
    content['description'] = content['description'].replace('\xa0','').replace('[...]', '')
    # print("DESCRIPTION: ", content['description'])
    
    cities = des.find('p', class_="ville").get_text().strip()
    content['city'] = " ".join(re.findall('[a-zA-ZÀ-ȕ]+', cities))
    # print("CITY: ", content['city'])
    
    content['postal_code'] = re.findall('[0-9]+', cities)[-1]
    # print("POSTAL CODE: ", content['postal_code'])
    
    content['price'] = int(des.find('p', class_="prix").find('strong').get_text().replace('\u202f','').replace('\xa0€','').split()[0])
    # print("PRICE: ", content['price'])
    
    content['type'] = des.find('h3').get_text().split()[0]
    # print("TYPE: ", content['type'])
    
    content['number_pieces'] = int(des.find('h3').find('strong').get_text().split()[0])
    # print("NUMBER PIECES: ", content['number_pieces'])
    
    content['surface'] = des.find('h3').get_text().split()[-1]
    # print("SURFACE: ", content['surface'])


    data_citya.append(content)

In [7]:
data_citya

[{'id': 0,
  'title': 'Appartement                     1 pièce 23m²',
  'description': "VENTE : appartement T1 (23 m²) à La Grande-Motte - STUDIO CABINE\nCOUCHANT PROCHE PLAGE. En vente : idéalement situé, découvrez \n                            cet appartement studio cabine  aux 23 m² optimisés avec vue dégagée à La Grande-Motte (34280). Il offre une cabine avec possibilité de lit en 140 cm, une cuisine équipée ouverte sur le séjour et une salle de bains avec WC. Cet appartement est équipé d'une climatisation réversible. Très bon état général.\nProfitez d'espace extérieur pour gagner en confort avec une terrasse d'une surface de 6 m².\nCet appartement se situe dans un immeuble avec ascenseur. Il est en excellent état général. Possibilité de garage en supplément. La sécurité du bâtiment est assurée par un digicode.\nL'appartement se situe dans un secteur attractif. Le Collège Philippe Lamour se trouve à moins de 10 minutes à pied. Niveau transports, il y a la gare Le Grau-du-Roi à moin

## Téléchargement données de plusieurs pages

In [8]:
page = 1
url = fr"https://www.citya.com/annonces/vente/appartement,maison?sort=b.dateMandat&direction=desc&page={page}"
response = requests.get(url)

while response.status_code == 200 :
    url = fr"https://www.citya.com/annonces/vente/appartement,maison?sort=b.dateMandat&direction=desc&page={page}"
    response = requests.get(url)
    html = response.content
    soup = bs(html, "html")
    # print("Status code: ", response.status_code)
    descs = soup.find_all('div', {'class': "infos"})
    # print("DESCS: ", descs)

    data_citya = []
        

    for i,des in enumerate(descs):
        content = {'id': i,
                    'title': None,
                    'description': None,
                    'type': None,
                    'surface': None,
                    'price': None,
                    'city': None,
                    'postal_code': None,
                    'number_pieces': None,
                    }
        
        # print("DES: ", des)
        content['title'] = des.find('h3').get_text().strip().replace('\n','')
        
        
        content['description'] = des.find('p', class_="description-start").get_text() + "\n" + \
                                des.find('span', class_="description-more").get_text()
        content['description'] = content['description'].replace('\xa0','')
        # print("DESCRIPTION: ", content['description'])
        
        
        cities = des.find('p', class_="ville").get_text().strip()
        
        content['city'] = " ".join(re.findall('[a-zA-ZÀ-ȕ]+', cities))
        # print("CITY: ", content['city'])
        
        content['postal_code'] = re.findall('[0-9]+', cities)[-1]
        # print("POSTAL CODE: ", content['postal_code'])
        
        content['price'] = int(des.find('p', class_="prix").find('strong').get_text().replace('\u202f','').replace('\xa0€','').split()[0])
        # print("PRICE: ", content['price'])
        
        content['type'] = des.find('h3').get_text().split()[0]
        # print("TYPE: ", content['type'])
        
        dt = des.find('h3').find('strong').get_text().replace('s','')
        # print("DT: ", dt)
        
        if "pièce" in dt and "m²" in dt:
            content['number_pieces'] = int(dt.split('pièce')[0])
            content['surface'] = float(dt.split('pièce')[1].split('m²')[0])
            # print("NUMBER PIECES: ", content['number_pieces'])
            # print("SURFACE: ", content['surface'])    
        elif "pièce" in dt and "m²" not in dt:
            content['number_pieces'] = int(dt.split('pièce')[0])
            # print("NUMBER PIECES: ", content['number_pieces'])  
        elif "pièce" not in dt and "m²" in dt:
            content['surface'] = float(dt.split('m²')[0])
            # print("SURFACE: ", content['surface'])


        data_citya.append(content)
        
    if data_citya == []:
        break
        
        
    page_path = rf"{PATH_DATA_RAW}/Citya/"

    nom_fichier = rf"{page_path}/json/data_page_{page}.json"
    if os.path.exists(nom_fichier):
        print(f'Le fichier data_page_{page}.json existe déja.\n')
    else: 
        with open(nom_fichier, "w", encoding="utf-8") as fichier:  
            json.dump(data_citya, fichier, ensure_ascii=False)
            fichier.close()
        print(f'Le fichier data_page_{page}.json a été créé avec succès.\n')
            
    
    parquet_path = rf"{page_path}/parquet/data_page_{page}.parquet"
    if os.path.exists(parquet_path):
        print(f'Le fichier data_page_{page}.parquet existe déjà.\n')
    else:
        # Convertir la liste en un DataFrame pandas
        df = pd.DataFrame(data_citya)

        # Enregistrer le DataFrame au format Parquet
        table = pa.Table.from_pandas(df)
        pq.write_table(table, parquet_path, compression='gzip')
        print(f'Le fichier data_page_{page}.parquet a été créé avec succès.\n')

        
        
    page += 1

Le fichier data_page_1.json a été créé avec succès.

Le fichier data_page_1.parquet a été créé avec succès.

Le fichier data_page_2.json a été créé avec succès.

Le fichier data_page_2.parquet a été créé avec succès.

Le fichier data_page_3.json a été créé avec succès.

Le fichier data_page_3.parquet a été créé avec succès.

Le fichier data_page_4.json a été créé avec succès.

Le fichier data_page_4.parquet a été créé avec succès.

Le fichier data_page_5.json a été créé avec succès.

Le fichier data_page_5.parquet a été créé avec succès.

Le fichier data_page_6.json a été créé avec succès.

Le fichier data_page_6.parquet a été créé avec succès.

Le fichier data_page_7.json a été créé avec succès.

Le fichier data_page_7.parquet a été créé avec succès.

Le fichier data_page_8.json a été créé avec succès.

Le fichier data_page_8.parquet a été créé avec succès.

Le fichier data_page_9.json a été créé avec succès.

Le fichier data_page_9.parquet a été créé avec succès.

Le fichier data_pag